# The TextAttack🐙 ecosystem: search, transformations, and constraints

An attack in TextAttack consists of three basic parts.

### Search method
The **search method** which explores the space of potential transformations and tries to locate a successful attack. Greedy search, beam search, and brute-force search are all examples of search methods. Since the search method is the backbone of the attack, the term "search" is often substituted with "attack method" or just "attack". In TextAttack, all three of those terms (search, attack, attack method) mean the same thing.

### Transformation
A **transformation** takes a text input and transforms it, replacing words or phrases with similar ones, while trying not to change the meaning. Paraphrase and synonym substitution are two broad classes of transformations.

### Constraints
Finally, **constraints** determine whether or not a given transformation is valid. Transformations don't perfectly preserve syntax or semantics, so additional constraints can increase the probability that these qualities are preserved from the source to adversarial example. There are many types of constraints: overlap constraints that measure edit distance, syntactical  constraints check part-of-speech and grammar errors, and semantic constraints like language models and sentence encoders.

# A custom transformation

This lesson explains how to create a custom transformation. In TextAttack, many transformations involve *word swaps*: they take a word and try and find suitable substitutes. Some attacks focus on replacing characters with neighboring characters induce "typos". (These don't intend to preserve the grammaticality of inputs.) Other attacks rely on semantics: they take a word and try to replace it with semantic equivalents.


### Banana word swap 🍌

As an introduction to writing transformations for TextAttack, we're going to try a very simple transformation: one that replaces any given word with the word 'banana'. In TextAttack, there's an abstract `WordSwap` class that handles the heavy lifting of breaking sentences into words and avoiding replacement of stopwords. We can extend `WordSwap` and implement a single method, `_get_replacement_words`, to indicate to replace each word with 'banana'.

In [1]:
from textattack.transformations.word_swap import WordSwap

class BananaWordSwap(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.
        
            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        return ['banana']

## Using our transformation

Now we have the transformation chosen, but we're missing a few other things. To complete the attack, we need to choose the **search method** and **constraints**. And to use the attack, we need a **goal function**, a **model** and a **dataset**. (The goal function indicates the task our model performs – in this case, classification – and the type of attack – in this case, we'll perform an untargeted attack.)

### Creating the goal function, model, and dataset
We are performing an untargeted attack on a classification model, so we'll use the `UntargetedClassification` class. For the model, let's use an LSTM trained for news classification on the AG News dataset. Luckily, TextAttack comes with 1000 text samples from some popular datasets, as well as pretrained models for those datasets. So we don't have to train our own model, or procure someone else's. We can just use the built-in datasets and models for this.

In [2]:
# Import the dataset.
from textattack.datasets.classification import AGNews
# Create the model.
from textattack.models.classification.lstm import LSTMForAGNewsClassification
model = LSTMForAGNewsClassification()
# Create the goal function using the model.
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model)

### Creating the attack
Let's keep it simple: let's use a greedy search method, and let's not use any constraints for now. 

In [22]:
from textattack.attack_methods import GreedyWordSwap

# We're going to use our Banana word swap class as the attack transformation.
transformation = BananaWordSwap() 
# And, we don't want to use any constraints.
constraints = []
# Now, let's make the attack using these parameters:
attack = GreedyWordSwap(goal_function, transformation, constraints)

### Using the attack

Let's use our attack to attack 10 samples (by setting `num_examples` to 10). Additionally, we set `attack_n` to `True`, which indicates that we should attack 10 samples, no matter what. If the model mispredicts a sample already, it isn't attacked; since `attack_n` is `True`, if a sample is mispredicted, we'll take try the next thing in the dataset, and continue until `num_examples` attacks have been completed.

In [5]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.

results_iterable = attack.attack_dataset(AGNews(), num_examples=10, attack_n=True)
results = []

logger = CSVLogger(color_method='html')

for result in tqdm(results_iterable, total=10):
    logger.log_attack_result(result)

CSVLogger exiting without calling flush().
100%|██████████| 10/10 [00:02<00:00,  4.61it/s]


### Visualizing attack results

We are logging `AttackResult` objects using a `CSVLogger`. This logger stores all attack results in a dataframe, which we can easily access and display. Since we set `color_method` to `'html'`, the attack results will display their differences, in color, in HTML. Using `IPython` utilities and `pandas`

In [21]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

from IPython.core.display import display, HTML
display(HTML(logger.df[['passage_1', 'passage_2']].to_html(escape=False)))

,passage_1,passage_2
0,"Thirst, Fear and Bribes on Desert Escape from Africa AGADEZ, Niger (Reuters) - Customs officers in this dusty Saharan town turned a blind eye as yet another creaking truck piled with grain, smuggled cigarettes and dozens of migrants heading for Europe rumbled off into the desert.","Thirst, Fear and Bribes on Desert Escape from Africa banana, Niger (Reuters) - banana officers in this dusty Saharan town turned a banana banana as banana another creaking truck piled with grain, smuggled cigarettes and dozens of migrants heading for Europe rumbled off into the desert."
1,"Toshiba 20 TV freaks out, sends distress signal See what happens when your warranty runs out?. In this case, a 20 Toshiba owned by Chris van Rossman started sending out the international distress signal at 121.","Toshiba 20 TV freaks out, sends distress signal See what banana when your warranty runs out?. In this case, a 20 Toshiba owned by Chris van Rossman started sending out the international distress signal at 121."
2,"British hostage tried fleeing before death: report The portrait of Ken Bigley, who was murdered in Iraq October 7, stands in front of the congregation during a service at Liverpool #39;s Roman Catholic Cathedral on October 10.","British hostage tried fleeing before death: report The portrait of Ken banana, who was murdered in Iraq October 7, stands in front of the congregation during a service at Liverpool #39;s Roman Catholic Cathedral on October 10."
3,"Keychain clicker kills TVs Discrete device turns off televisions, creating a little peace and quiet. Until the yelling starts.","banana banana kills banana banana banana banana off banana, banana a banana banana and banana. Until the yelling starts."
4,"Cleric returns to Iraq, orders march on Najaf Powerful Shiite leader says he plans to lead a mass demonstration today to end fighting. Iraqi hostage: Militants said Wednesday they had kidnapped the brother-in-law of Iraqi Defense Minister Hazem Shaalan","banana banana to banana, banana banana on banana banana banana banana banana he banana to banana a banana banana banana to banana banana. banana banana: Militants banana banana they had banana the banana-in-banana of banana banana banana banana banana"
5,"Hewitt Beats Roddick to Reach Masters Final HOUSTON (Reuters) - A fired-up Lleyton Hewitt defused hard-hitting Andy Roddick 6-3, 6-2 on Saturday, scurrying into the final of the Masters Cup for the third time in four years.","Hewitt Beats Roddick to banana banana Final HOUSTON (Reuters) - A fired-up banana banana banana banana-banana Andy Roddick 6-3, 6-2 on banana, scurrying into the final of the Masters Cup for the third time in four years."
6,"Despite booming economy, no concrete move on debt relief (AFP) AFP - Senior finance officials have hailed a robust global economic recovery, albeit one threatened by surging oil prices, but made little headway pushing China toward currency reform and took no firm steps to ease the debt of the world's poorest nations.","banana banana banana, no banana banana on banana banana (AFP) AFP - Senior finance officials have hailed a robust global economic recovery, albeit one threatened by surging oil banana, but banana banana banana banana banana banana banana banana and banana no firm steps to ease the debt of the world's poorest nations."
7,"Just a close call for closer No need to check with your primary care provider about the origin of that nervous tic you woke up with this morning, on the first full day of autumn. Ninth-inning home runs allowed in three consecutive games by Sox closer Keith Foulke, who also was tagged with blown saves in each of the last two games, were enough to leave ...","Just a banana call for closer No banana to banana with your primary care provider about the banana of that banana tic you woke up with this banana, on the first full day of autumn. Ninth-inning home runs allowed in three consecutive games by Sox banana Keith banana, who also was tagged with blo

## Conclusion 🍌

We can examine these examples for a good idea of how many words had to be changed to "banana" to change the prediction score from the correct class to another class. Looks like some examples needed only a single "banana", while others needed up to 17 "banana" substitutions to change the class score. Wow!